<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# `pandas` Data Munging Overview: Part 2


---

### Lesson Guide
- [Exercise #3](#exercise-3)
- [Split-Apply-Combine](#split-apply-combine)
    - [`.groupby()`](#groupby)
    - [Apply Functions to Groups and Combine](#apply-combine)
- [Exercise #4](#exercise-4)
- [Indexing](#indexing)
    - [Location Indexing With `.loc()`](#loc)
    - [Position Indexing With `.iloc()`](#iloc)
- [Other Frequently Used Features](#frequent)
    - [Using Map Functions With Replacement Dictionaries](#map-dict)
    - [Encoding Strings as Integers With `.factorize()`](#factorize)
    - [Determining Unique Values](#unique)
    - [Replacing Values With `.replace()`](#replace)
    - [Series String Methods With `.str`](#series-str)
    - [Datetime Conversion and Arithmetic](#datetime)
    - [Setting and Resetting the Index](#set-reset-index)
    - [Sorting by Index](#sort-by-index)
    - [Changing the Data Type of a Column](#change-dtype)
    - [Creating Dummy-Coded Columns](#dummy)
    - [Concatenating DataFrames](#concatenate)
    - [Detecting and Dropping Duplicate Rows](#duplicate-rows)
    - [Writing a DataFrame to a `.csv`](#write-csv)
    - [Pickling a DataFrame](#pickle)
    - [Randomly Sampling a DataFrame](#sample)
- [Infrequently Used Features](#infrequent)
    - [Creating DataFrames From Dictionaries and Lists of Lists](#toy-dataframes)
    - [Performing Cross-Tabulations](#crosstab)
    - [Query-Filtering Syntax](#query)
    - [Calculating Memory Usage](#memory-usage)
    - [Converting Column to Category Type](#category-type)
    - [Creating Columns With `.assign()`](#assign)
    - [Limiting the Number of Rows to Load in a File Read](#limit-rows-read)
    - [Manually Setting the Number of Rows and Columns to Print](#manual-print)

In [1]:
import pandas as pd
import numpy as np


<a id='exercise-3'></a>
## Exercise #3

---

**Using the UFO data provided below:**
1. Read in the data.
2. Check the shape and describe the columns.
3. Find the four most frequently reported colors.
4. Find the most frequent city for reports in state `VA`.
5. Find only UFO reports from Arlington, VA.
6. Find the number of missing values in each column.
7. Show only UFO reports where `city` is missing.
8. Count the number of rows with no null values.
9. Amend column names with spaces to have underscores.
10. Make a new column that is a combination of `city` and `state`.

In [2]:
ufo_csv = '../../../../resource-datasets/ufo_sightings/ufo.csv'

In [3]:
# A:
ufo = pd.read_csv(ufo_csv)

In [4]:
ufo.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00


<a id='split-apply-combine'></a>
## Split-Apply-Combine

---

![](assets/split_apply_combine.png)

<a id='groupby'></a>
### `.groupby()`

**Q.1** Using the `drinks` DataFrame, calculate the mean `beer` servings by continent.

In [5]:
drinks = pd.read_csv('../../../../resource-datasets/alcohol_by_country/drinks.csv')
column_names = ['country', 'beer', 'spirit', 'wine', 'liters', 'continent']
drinks.columns = column_names

In [6]:
# A:
drinks['beer'].mean()

106.16062176165804

In [7]:
drinks.head()

,country,beer,spirit,wine,liters,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


**Q.2** Describe the `beer` column by continent.

In [8]:
# A:
groupbyContinent = drinks.groupby('continent')
groupbyContinent['beer'].describe()

,count,mean,std,min,25%,50%,75%,max
continent,,,,,,,,
AF,53.0,61.471698,80.557816,0.0,15.00,32.0,76.00,376.0
AS,44.0,37.045455,49.469725,0.0,4.25,17.5,60.50,247.0
EU,45.0,193.777778,99.631569,0.0,127.00,219.0,270.00,361.0
OC,16.0,89.687500,96.641412,0.0,21.00,52.5,125.75,306.0
SA,12.0,175.083333,65.242845,93.0,129.50,162.5,198.00,333.0


<a id='apply-combine'></a>
### Apply Functions to Groups and Combine

**Q.1** Find the `count`, `mean`, `minimum`, and `maximum `of the `beer` column by continent.

In [9]:
# A:
print('mean :' , groupbyContinent['beer'].mean())
print('count :' , groupbyContinent['beer'].count())
print('min :' ,groupbyContinent['beer'].min())
print('mx :' ,groupbyContinent['beer'].max())

mean : continent
AF     61.471698
AS     37.045455
EU    193.777778
OC     89.687500
SA    175.083333
Name: beer, dtype: float64
count : continent
AF    53
AS    44
EU    45
OC    16
SA    12
Name: beer, dtype: int64
min : continent
AF     0
AS     0
EU     0
OC     0
SA    93
Name: beer, dtype: int64
mx : continent
AF    376
AS    247
EU    361
OC    306
SA    333
Name: beer, dtype: int64


**Q.2** Perform the same task as in Q.1, but now sort the output by the `mean` column.

In [10]:
# A:
groupbyContinent = drinks.groupby('continent' , sort=False)
groupbyContinent.beer.describe().sort_values('mean' , ascending=False)



,count,mean,std,min,25%,50%,75%,max
continent,,,,,,,,
EU,45.0,193.777778,99.631569,0.0,127.00,219.0,270.00,361.0
SA,12.0,175.083333,65.242845,93.0,129.50,162.5,198.00,333.0
OC,16.0,89.687500,96.641412,0.0,21.00,52.5,125.75,306.0
AF,53.0,61.471698,80.557816,0.0,15.00,32.0,76.00,376.0
AS,44.0,37.045455,49.469725,0.0,4.25,17.5,60.50,247.0


**Q.3** Apply a custom function to all columns of the `drinks` DataFrame, grouping by continent.

In [11]:
# A:
drinks.groupby('continent').sum()

,beer,spirit,wine,liters
continent,,,,
AF,3258,866,862,159.4
AS,1630,2677,399,95.5
EU,8720,5965,6400,387.8
OC,1435,935,570,54.1
SA,2101,1377,749,75.7


**Q.4** **Note:** If you don't specify a column for the aggregation function, it will be applied to all numeric columns.

<a id='exercise-4'></a>

## Exercise #4

---

**Using the `users` DataFrame**:
1. Count the number of distinct occupations in `users`.
2. Calculate the mean age by occupation.
3. Calculate the minimum and maximum age by occupation.
4. Calculate the mean age by cross-sections of `occupation` and `gender`.

> **Tip**: Multiple columns can be passed to the `.groupby()` function for more granular cross-sections.

In [12]:
users = pd.read_table('../../../../resource-datasets/users/users.txt', sep= '|')

In [13]:
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [14]:
# A:
len(users.occupation.unique())

21

In [15]:
users.groupby('occupation').age.mean()

occupation
administrator    38.746835
artist           31.392857
doctor           43.571429
educator         42.010526
engineer         36.388060
entertainment    29.222222
executive        38.718750
healthcare       41.562500
homemaker        32.571429
lawyer           36.750000
librarian        40.000000
marketing        37.615385
none             26.555556
other            34.523810
programmer       33.121212
retired          63.071429
salesman         35.666667
scientist        35.548387
student          22.081633
technician       33.148148
writer           36.311111
Name: age, dtype: float64

In [16]:
users.groupby('occupation').age.agg(['min', 'max'])

,min,max
occupation,,
administrator,21,70
artist,19,48
doctor,28,64
educator,23,63
engineer,22,70
entertainment,15,50
executive,22,69
healthcare,22,62
homemaker,20,50


In [17]:
users.groupby(['occupation', 'gender']).age.mean()

occupation     gender
administrator  F         40.638889
               M         37.162791
artist         F         30.307692
               M         32.333333
doctor         M         43.571429
educator       F         39.115385
               M         43.101449
engineer       F         29.500000
               M         36.600000
entertainment  F         31.000000
               M         29.000000
executive      F         44.000000
               M         38.172414
healthcare     F         39.818182
               M         45.400000
homemaker      F         34.166667
               M         23.000000
lawyer         F         39.500000
               M         36.200000
librarian      F         40.000000
               M         40.000000
marketing      F         37.200000
               M         37.875000
none           F         36.500000
               M         18.600000
other          F         35.472222
               M         34.028986
programmer     F         32.16666

<a id='indexing'></a>
## Indexing

---
<a id='loc'></a>
### Location Indexing With `.loc()`

**Q.1** Select all rows and the `city` column from the UFO data set using `.loc()`.

In [18]:
ufo.loc[:, 'City']  

0                      Ithaca
1                 Willingboro
2                     Holyoke
3                     Abilene
4        New York Worlds Fair
5                 Valley City
6                 Crater Lake
7                        Alma
8                     Eklutna
9                     Hubbard
10                    Fontana
11                   Waterloo
12                     Belton
13                     Keokuk
14                  Ludington
15                Forest Home
16                Los Angeles
17                  Hapeville
18                     Oneida
19                 Bering Sea
20                   Nebraska
21                        NaN
22                        NaN
23                  Owensboro
24                 Wilderness
25                  San Diego
26                 Wilderness
27                     Clovis
28                 Los Alamos
29               Ft. Duschene
                 ...         
80513              Manahawkin
80514             New Bedford
80515     

**Q.2** Select all rows and columns in `city` and `state`.

In [19]:
# A:
ufo.loc[:, ['City', 'State']]  

,City,State
0,Ithaca,NY
1,Willingboro,NJ
2,Holyoke,CO
3,Abilene,KS
4,New York Worlds Fair,NY
5,Valley City,ND
6,Crater Lake,CA
7,Alma,MI
8,Eklutna,AK
9,Hubbard,OR


**Q.3** Select all rows and columns from `city` *through* `state`.

In [20]:
# A:
ufo.loc[:, 'City':'State'] 

,City,Colors Reported,Shape Reported,State
0,Ithaca,NaN,TRIANGLE,NY
1,Willingboro,NaN,OTHER,NJ
2,Holyoke,NaN,OVAL,CO
3,Abilene,NaN,DISK,KS
4,New York Worlds Fair,NaN,LIGHT,NY
5,Valley City,NaN,DISK,ND
6,Crater Lake,NaN,CIRCLE,CA
7,Alma,NaN,DISK,MI
8,Eklutna,NaN,CIGAR,AK
9,Hubbard,NaN,CYLINDER,OR


**Q.4** Select:
- All columns at row 0.
- All columns at rows 0:2.
- Columns `city` through `state` at rows 0:2.

In [21]:
# A:
ufo.loc[0, :]                   
ufo.loc[0:2, :]               
ufo.loc[0:2, 'City':'State']

,City,Colors Reported,Shape Reported,State
0,Ithaca,NaN,TRIANGLE,NY
1,Willingboro,NaN,OTHER,NJ
2,Holyoke,NaN,OVAL,CO


<a id='iloc'></a>
### Position indexing with `.iloc`

**Q.1** Select all rows and columns in position 0 and 3.

In [22]:
# A:
ufo.iloc[:, [0, 3]]

,City,State
0,Ithaca,NY
1,Willingboro,NJ
2,Holyoke,CO
3,Abilene,KS
4,New York Worlds Fair,NY
5,Valley City,ND
6,Crater Lake,CA
7,Alma,MI
8,Eklutna,AK
9,Hubbard,OR


**Q.2** Select all rows and columns in positions 0 through 4.

In [23]:
# A:
ufo.iloc[:, 0:4]

,City,Colors Reported,Shape Reported,State
0,Ithaca,NaN,TRIANGLE,NY
1,Willingboro,NaN,OTHER,NJ
2,Holyoke,NaN,OVAL,CO
3,Abilene,NaN,DISK,KS
4,New York Worlds Fair,NaN,LIGHT,NY
5,Valley City,NaN,DISK,ND
6,Crater Lake,NaN,CIRCLE,CA
7,Alma,NaN,DISK,MI
8,Eklutna,NaN,CIGAR,AK
9,Hubbard,NaN,CYLINDER,OR


**Q.3** Select rows in positions 0:3, along with all columns.

In [24]:
# A:
ufo.iloc[0:3, :]

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00


<a id='frequent'></a>
## Frequently Used Features

---
<a id='map-dict'></a>
### Using Map Functions With Replacement Dictionaries

In [25]:
# A:
df = pd.DataFrame({'one': {'A': 10, 'B': 20, 'C': 30, 'D': 40, 'E': 50}})
map_dict = {'A': 'every', 'B': 'good', 'C': 'boy', 'D': 'does', 'E': 'fine'}

df['one'] = df.index.to_series().map(map_dict)

df

,one
A,every
B,good
C,boy
D,does
E,fine


<a id='factorize'></a>
### Encoding Strings as Integers With `.factorize()`

In [26]:
# A:
labels , unique = pd.factorize(['b', 'b', 'c'])
labels

array([0, 0, 1])

<a id='unique'></a>
### Determining Unique Values

In [27]:
# A:
unique

array(['b', 'c'], dtype=object)

<a id='replace'></a>
### Replacing Values With `.replace()`

In [28]:
# A:
'Yasmin'.replace('Y',"J")

'Jasmin'

<a id='series-str'></a>
### Series String Methods With `.str`

In [29]:
# A:
s_str = pd.Series(['A', 'B', 'C'])

In [30]:
s_str.str.lower()

0    a
1    b
2    c
dtype: object

In [31]:
s_str.str.upper()

0    A
1    B
2    C
dtype: object

In [32]:
s_str.str.len()

0    1
1    1
2    1
dtype: int64

<a id='datetime'></a>
### Datetime Conversion and Arithmetic

In [33]:
# A:
import datetime
myBD = '17/1/1994'
d = datetime.datetime.strptime(myBD, '%d/%m/%Y') + datetime.timedelta(days=0)
print(d.strftime('%d/%m/%Y'))

17/01/1994


<a id='set-reset-index'></a>
### Setting and Resetting the Index

In [34]:
# A:
ufo.reset_index()


,index,City,Colors Reported,Shape Reported,State,Time
0,0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
5,5,Valley City,NaN,DISK,ND,9/15/1934 15:30
6,6,Crater Lake,NaN,CIRCLE,CA,6/15/1935 0:00
7,7,Alma,NaN,DISK,MI,7/15/1936 0:00
8,8,Eklutna,NaN,CIGAR,AK,10/15/1936 17:00
9,9,Hubbard,NaN,CYLINDER,OR,6/15/1937 0:00


In [35]:
ufo.set_index('Time')

,City,Colors Reported,Shape Reported,State
Time,,,,
6/1/1930 22:00,Ithaca,NaN,TRIANGLE,NY
6/30/1930 20:00,Willingboro,NaN,OTHER,NJ
2/15/1931 14:00,Holyoke,NaN,OVAL,CO
6/1/1931 13:00,Abilene,NaN,DISK,KS
4/18/1933 19:00,New York Worlds Fair,NaN,LIGHT,NY
9/15/1934 15:30,Valley City,NaN,DISK,ND
6/15/1935 0:00,Crater Lake,NaN,CIRCLE,CA
7/15/1936 0:00,Alma,NaN,DISK,MI
10/15/1936 17:00,Eklutna,NaN,CIGAR,AK


<a id='sort-by-index'></a>
### Sorting by Index

In [36]:
# A:
ufo.sort_index()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
5,Valley City,NaN,DISK,ND,9/15/1934 15:30
6,Crater Lake,NaN,CIRCLE,CA,6/15/1935 0:00
7,Alma,NaN,DISK,MI,7/15/1936 0:00
8,Eklutna,NaN,CIGAR,AK,10/15/1936 17:00
9,Hubbard,NaN,CYLINDER,OR,6/15/1937 0:00


<a id='change-dtype'></a>
### Changing the Data Type of a Column

In [37]:
# A:
import datetime as dt  
ufo['Time'].dtypes
ufo['Time'] = pd.to_datetime(ufo['Time'])
ufo['Time'].dtypes

dtype('<M8[ns]')

<a id='dummy'></a>
### Creating Dummy-Coded Columns

In [38]:
# A:
dummy = pd.Series(['a','v','d'])
pd.get_dummies(dummy)

,a,d,v
0,1,0,0
1,0,0,1
2,0,1,0


<a id='concatenate'></a>
### Concatenating DataFrames

In [39]:
# A:
result = pd.concat([dummy,s_str], keys=['a', 'v', 'd'])
result

a  0    a
   1    v
   2    d
v  0    A
   1    B
   2    C
dtype: object

<a id='duplicate-rows'></a>
### Detecting and Dropping Duplicate Rows

In [40]:
# A:
result.drop_duplicates()
result

a  0    a
   1    v
   2    d
v  0    A
   1    B
   2    C
dtype: object

<a id='write-csv'></a>
### Writing a DataFrame to a `.csv`
```python
# Write a DataFrame out to a `.csv`.
drinks.to_csv('drinks_updated.csv')  # Index is used as the first column
drinks.to_csv('drinks_updated.csv', index=False) # Ignore index
```

<a id='pickle'></a>
### Pickling a DataFrame
```python
# Save a DataFrame to disk (a.k.a., "pickle") and read it from disk (a.k.a., "unpickle").
drinks.to_pickle('drinks_pickle')
pd.read_pickle('drinks_pickle')
```

<a id='sample'></a>
### Randomly Sampling a DataFrame

In [41]:
# A:
result.sample(frac=0.2)

a  1    v
dtype: object

<a id='infrequent'></a>
## Infrequently Used Features

---

<a id='toy-dataframes'></a>
### Creating DataFrames From Dictionaries and Lists of Lists

In [42]:
# A: From Dict
test_dict = [{'one':'A','Two':'B'}]
test_dict_DF = pd.DataFrame(test_dict)
test_dict_DF.head()

,Two,one
0,B,A


In [43]:
# A:
test_list = [['one','A'],['Two','B']]
test_list_DF = pd.DataFrame(test_list)
test_list_DF.head()

,0,1
0,one,A
1,Two,B


<a id='crosstab'></a>
### Performing Cross-Tabulations

In [44]:
# A:
list1 = ['a','b','c']
list2 = ['d','f','g']
list3 = ['h','e','k']
pd.crosstab(list1, [list2, list3])

,,col_0,d,f,g
,,col_1,h,e,k
row_0,row_1,row_2,,,
a,b,c,1,1,1


<a id='query'></a>
### Query-Filtering Syntax

In [62]:
# A:
ufo.query('State == "NY"')

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00
41,New York City,NaN,DISK,NY,1946-01-08 02:00:00
50,Syracuse,NaN,DISK,NY,1947-02-20 22:15:00
74,Syracuse,NaN,CIGAR,NY,1947-07-14 10:30:00
87,Saranac Lake,NaN,DISK,NY,1947-08-12 22:00:00
96,New York City,NaN,CIRCLE,NY,1948-08-01 02:00:00
119,New York City,NaN,OVAL,NY,1950-01-02 00:00:00
158,Jackson Heights,NaN,DISK,NY,1951-06-30 07:00:00
171,Binghamton,NaN,DISK,NY,1952-02-10 21:00:00


<a id='memory-usage'></a>
### Calculating Memory Usage

In [47]:
# A:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

43941888


<a id='category-type'></a>
### Converting Column to Category Type

In [55]:
# A:
ufo['Time'] = ufo['Time'].astype('category')
ufo['Time'].dtypes

CategoricalDtype(categories=['1930-06-01 22:00:00', '1930-06-30 20:00:00',
                  '1931-02-15 14:00:00', '1931-06-01 13:00:00',
                  '1933-04-18 19:00:00', '1934-09-15 15:30:00',
                  '1935-06-15 00:00:00', '1936-07-15 00:00:00',
                  '1936-10-15 17:00:00', '1937-06-15 00:00:00',
                  ...
                  '2014-09-04 22:00:00', '2014-09-04 22:05:00',
                  '2014-09-04 22:15:00', '2014-09-04 22:30:00',
                  '2014-09-04 23:00:00', '2014-09-04 23:20:00',
                  '2014-09-05 01:14:00', '2014-09-05 02:40:00',
                  '2014-09-05 03:43:00', '2014-09-05 05:30:00'],
                 ordered=False)

<a id='assign'></a>
### Creating Columns With `.assign()`

In [60]:
# A:
df = pd.DataFrame({'A':['a','b'], 'B': ['c','d']})
df.head()

,A,B
0,a,c
1,b,d


In [61]:
df.assign(D=['e','f'])

,A,B,D
0,a,c,e
1,b,d,f


<a id='limit-rows-read'></a>
### Limiting the Number of Rows to Load in a File Read

In [64]:
# A:
users = pd.read_table('../../../../resource-datasets/users/users.txt', sep= '|' , nrows = 3)
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067


<a id='manual-print'></a>
### Manually Setting the Number of Rows and Columns to Print

In [65]:
# A:
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)